## Exercise 2: Train, Test,

In [1]:
import torch
import random
from torch.nn import functional as F

In [ ]:
words = open('../names.txt').read().splitlines()

In [3]:
characters = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(characters)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
stoi, itos

({'a': 1,
  'b': 2,
  'c': 3,
  'd': 4,
  'e': 5,
  'f': 6,
  'g': 7,
  'h': 8,
  'i': 9,
  'j': 10,
  'k': 11,
  'l': 12,
  'm': 13,
  'n': 14,
  'o': 15,
  'p': 16,
  'q': 17,
  'r': 18,
  's': 19,
  't': 20,
  'u': 21,
  'v': 22,
  'w': 23,
  'x': 24,
  'y': 25,
  'z': 26,
  '.': 0},
 {1: 'a',
  2: 'b',
  3: 'c',
  4: 'd',
  5: 'e',
  6: 'f',
  7: 'g',
  8: 'h',
  9: 'i',
  10: 'j',
  11: 'k',
  12: 'l',
  13: 'm',
  14: 'n',
  15: 'o',
  16: 'p',
  17: 'q',
  18: 'r',
  19: 's',
  20: 't',
  21: 'u',
  22: 'v',
  23: 'w',
  24: 'x',
  25: 'y',
  26: 'z',
  0: '.'})

In [4]:

random.shuffle(words)
num = len(words)
train, dev, test = words[:int(
    0.8*num)], words[int(0.8*num):int(0.9*num)], words[int(0.9*num):]
len(train) + len(dev) + len(test) == num

True

In [5]:
N = torch.zeros(27, 27, 27, dtype=torch.int32)
g = torch.Generator().manual_seed(42)

for word in train:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        idx3 = stoi[ch3]
        N[idx1, idx2, idx3] += 1

In [6]:
xs, ys = [], []

for word in train:
    chs = '.' + word + '.'
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[3:]):
        idx1, idx2, idx3 = stoi[ch1], stoi[ch2], stoi[ch3]
        xs.append([idx1, idx2])
        ys.append(idx3)


xs, ys = torch.tensor(xs), torch.tensor(ys)
# num = ys.shape[0]
W = torch.rand((27*2, 27), generator=g, requires_grad=True)
xs.shape, ys.shape

(torch.Size([131143, 2]), torch.Size([131143]))

In [7]:
n = 1000
for i in range(n+1):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc.view(-1, 2*27) @ W
    counts = torch.exp(logits)
    # print(counts.shape)
    probs = counts/counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() - 0.1 * W.mean()

    if i % 100 == 0 or i == 0 or i == n:
        print(f"step : {i}  loss : {loss.item()}")

    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

step : 0  loss : 3.376685619354248
step : 100  loss : 2.393401622772217
step : 200  loss : 2.3531460762023926
step : 300  loss : 2.3169102668762207
step : 400  loss : 2.281663656234741
step : 500  loss : 2.2468087673187256
step : 600  loss : 2.212146520614624
step : 700  loss : 2.177593469619751
step : 800  loss : 2.1431078910827637
step : 900  loss : 2.108666181564331
step : 1000  loss : 2.0742554664611816


In [8]:
def prepare_data(words: list[str], stoi: dict) -> torch.tensor:
    xs, ys = [], []
    for word in words:
        chs = '.' + word + '.'
        for ch1, ch2, ch3 in zip(chs, chs[1:], chs[3:]):
            idx1, idx2, idx3 = stoi[ch1], stoi[ch2], stoi[ch3]
            xs.append([idx1, idx2])
            ys.append(idx3)
    return torch.tensor(xs), torch.tensor(ys)


xs_test, ys_test = prepare_data(test, stoi=stoi)
xs_dev, ys_dev = prepare_data(dev, stoi)

In [9]:
xenc = F.one_hot(xs_dev, num_classes=27).float()
logits = xenc.view(-1, 2*27) @ W
counts = torch.exp(logits)
probs = counts/counts.sum(dim=1, keepdim=True)
loss = -probs[torch.arange(ys_dev.shape[0]),
              ys_dev].log().mean() - 0.1 * W.mean()
print(loss.item())

2.0833022594451904


In [10]:
xenc = F.one_hot(xs_test, num_classes=27).float()
logits = xenc.view(-1, 2*27) @ W
counts = torch.exp(logits)
probs = counts/counts.sum(dim=1, keepdim=True)
loss = -probs[torch.arange(ys_test.shape[0]),
              ys_test].log().mean() - 0.1 * W.mean()
print(loss.item())

2.070448875427246
